In [1]:
import re 
import datetime
import pandas as pd

import urllib.request as urllib_request
from urllib.error import URLError, HTTPError

from fiis_scraping.utils.helpers import *
from fiis_scraping.utils.functions import *
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', None)

In [2]:
URL = 'https://fiis.com.br/lista-de-fundos-imobiliarios/'
soup = create_bsoup_object(URL)

In [3]:
tickers = get_tickers(soup)

In [4]:
dataset = create_fiis_dataframe(tickers[0:10])

ABCP11
AFHI11
AGRX11
AIEC11
ALMI11
ALZM11
ALZR11
ANCR11B
APTO11
ARCT11


In [5]:
COLUMNS_LIST = ['ultimo_rendimento',
                'valor_patrimonial_por_cota',
                'numero_cotas',
                'numero_cotistas',
                'cotacao',
                'min_52_weeks',
                'max_52_weeks']
dataset = convert_string_to_float(dataset, COLUMNS_LIST)
dataset = convert_percentages_to_float(dataset, ['dividend_yield', 'return_12_months'])
dataset = break_tipo_FII(dataset)
dataset['registro_CVM'] = dataset['registro_CVM'].apply(lambda x: str(convert_to_datetime(x)))
#dataset["p_vp"] = (dataset['cotacao'] / dataset['valor_patrimonial_por_cota']).replace(np.inf, np.nan)
dataset['date'] = str(datetime.date.today() - datetime.timedelta(days=1))
dataset.head()

,ticker,administrador,dividend_yield,ultimo_rendimento,patrimonio_liquido,valor_patrimonial_por_cota,telefone,email,site,name,tipo_ANBIMA,registro_CVM,numero_cotas,numero_cotistas,CNPJ,cotacao,min_52_weeks,max_52_weeks,return_12_months,tipo,subtipo,date
0,ABCP11,Rio Bravo Investimentos,0.0071,0.52,"R$1,12 B",91.77,(11) 3509-6600,[email protected],www.riobravo.com.br,FII ABC IMOB,Renda Gestão Passiva,1996-05-15 00:00:00,12203832.0,19420.0,01.201.140/0001-90,73.61,60.01,78.27,0.0770,Tijolo,Shoppings,2022-09-30
1,AFHI11,BTG Pactual,0.0119,1.15,"R$287,45 M",95.20,21 3262-9600,[email protected],www.btgpactual.com,FII AFHI CRI,Títulos e Valores Mobiliários Gestão Ativa,2020-10-20 00:00:00,3019453.0,13980.0,36.642.293/0001-58,97.00,92.00,107.00,0.0543,Papel,CRIs,2022-09-30
2,AGRX11,Banco Genial,0.0125,0.13,"R$48,14 M",10.19,(21) 3923-3000,[email protected],http://www.bancogenial.com/,FIAGRO AGRX,Renda Gestão Ativa,2021-11-26 00:00:00,4725648.0,214.0,43.951.911/0001-07,10.36,9.80,11.11,0.0157,FIAGRO,FIAGRO,2022-09-30
3,AIEC11,MAF DTVM,0.0092,0.73,"R$489,18 M",101.38,3223-7700,[email protected],www.modal.com.br,FII AUTONOMY,Renda Gestão Ativa,2020-08-24 00:00:00,4824987.0,9877.0,35.765.826/0001-26,79.07,67.50,85.94,0.0054,Tijolo,Escritórios,2022-09-30
4,ALMI11,BTG Pactual,0.0004,0.37,"R$250,16 M",2250.09,21 3262-9600,[email protected],www.btgpactual.com,FII TORRE AL,Renda Gestão Passiva,2006-01-03 00:00:00,111177.0,2778.0,07.122.725/0001-00,910.00,830.00,1184.00,0.0421,Tijolo,Escritórios,2022-09-30


In [ ]:
URL = 'https://fiis.com.br/ifix/'
df_ifix = create_ifix_dataframe(URL)
COLUMNS_LIST = ['peso']
df_ifix = convert_percentages_to_float(df_ifix, COLUMNS_LIST)
df_ifix['peso'] = df_ifix['peso'].round(4)
df_ifix['date'] = str(datetime.date.today())
df_ifix.head()

In [ ]:
df_ifix.info()

## DynamoDB

### Sending the fiis dataframe

In [25]:
import boto3
import json
from decimal import Decimal

In [15]:
ticker_data_dict = dataset.to_dict(orient='records')

NameError: name 'dataset' is not defined

In [26]:
dynamodb_resource = boto3.resource("dynamodb")

In [14]:
batch_write(dynamodb_resource, "fiis", ticker_data_dict, ['ticker', 'date'])

TypeError: Object of type Decimal is not JSON serializable

In [27]:
table = dynamodb_resource.Table("fiis")
response = table.scan()
df = pd.DataFrame(response['Items'])
df.shape

(381, 22)

In [28]:
df.date.value_counts(dropna=False)

2022-09-30    381
Name: date, dtype: int64

In [29]:
df.head()

,patrimonio_liquido,administrador,subtipo,ticker,return_12_months,telefone,site,CNPJ,email,registro_CVM,name,ultimo_rendimento,tipo,date,cotacao,numero_cotistas,max_52_weeks,valor_patrimonial_por_cota,numero_cotas,tipo_ANBIMA,dividend_yield,min_52_weeks
0,"R$1,36 M",Oliveira Trust,Desenvolvimento,LATR11B,0.8339,(21) 3514-0000,www.oliveiratrust.com.br,17.209.378/0001-00,[email protected],2012-11-19 00:00:00,FII LATERES,14.5,Tijolo,2022-09-30,15.95,166,150,10.98,124104,Desenvolvimento para Venda Gestão Ativa,0.9093,15.95
1,"R$424,46 M",Hedge Investments DTVM LTDA,CRIs,HREC11,0.0311,(11) 3124-4100,www.hedgeinvest.com.br,35.507.262/0001-21,[email protected],2019-11-21 00:00:00,FII HEDGEREC,0.8,Papel,2022-09-30,93.97,468,99.88,91.15,4656688,Títulos e Valores Mobiliários Gestão Ativa,0.0085,90.9
2,"R$75,30 M",Banco Genial,Galpões,PLOG11,0.0192,(21) 3923-3000,http://www.bancogenial.com/,36.501.198/0001-34,[email protected],2020-12-01 00:00:00,FII PLURAL L,0.75,Tijolo,2022-09-30,79.74,2907,82.96,100.4,750000,Renda Gestão Ativa,0.0094,65.41
3,"R$173,10 M",BRL Trust DTVM,CRIs,VCRI11,0.0955,(11)31330350,www.brltrust.com.br,41.081.374/0001-66,[email protected],2021-03-23 00:00:00,FII VINCI CR,0.12,Papel,2022-09-30,9,3247,10.69,9.74,17773297,Títulos e Valores Mobiliários Gestão Ativa,0.0133,8.9
4,"R$58,21 M",Oliveira Trust,Escritórios,ERPA11,0.0154,(21) 3514-0000,www.oliveiratrust.com.br,31.469.385/0001-64,[email protected],2019-02-25 00:00:00,FII EUROPA,0.7,Tijolo,2022-09-30,128,350,140,116.21,500922,Renda Gestão Passiva,0.0054,110


In [20]:
df['dividend_yield'] = df['dividend_yield'].apply(float).round(4)
df['return_12_months'] = df['return_12_months'].apply(float).round(4)
#df = convert_percentages_to_float(df, ['dividend_yield', 'return_12_months'])

In [21]:
df.head()

,patrimonio_liquido,administrador,subtipo,ticker,return_12_months,telefone,site,CNPJ,email,registro_CVM,name,ultimo_rendimento,tipo,date,cotacao,numero_cotistas,max_52_weeks,valor_patrimonial_por_cota,numero_cotas,tipo_ANBIMA,dividend_yield,min_52_weeks
0,"R$1,36 M",Oliveira Trust,Desenvolvimento,LATR11B,0.8339,(21) 3514-0000,www.oliveiratrust.com.br,17.209.378/0001-00,[email protected],2012-11-19 00:00:00,FII LATERES,14.5,Tijolo,2022-09-29,15.95,166,150,10.98,124104,Desenvolvimento para Venda Gestão Ativa,0.9093,15.95
1,"R$1,36 M",Oliveira Trust,Desenvolvimento,LATR11B,0.8339,(21) 3514-0000,www.oliveiratrust.com.br,17.209.378/0001-00,[email protected],2012-11-19 00:00:00,FII LATERES,14.5,Tijolo,2022-09-30,15.95,166,150,10.98,124104,Desenvolvimento para Venda Gestão Ativa,0.9093,15.95
2,"R$424,46 M",Hedge Investments DTVM LTDA,CRIs,HREC11,0.0408,(11) 3124-4100,www.hedgeinvest.com.br,35.507.262/0001-21,[email protected],2019-11-21 00:00:00,FII HEDGEREC,0.8,Papel,2022-09-29,94,468,99.88,91.15,4656688,Títulos e Valores Mobiliários Gestão Ativa,0.0085,90.9
3,"R$424,46 M",Hedge Investments DTVM LTDA,CRIs,HREC11,0.0311,(11) 3124-4100,www.hedgeinvest.com.br,35.507.262/0001-21,[email protected],2019-11-21 00:00:00,FII HEDGEREC,0.8,Papel,2022-09-30,93.97,468,99.88,91.15,4656688,Títulos e Valores Mobiliários Gestão Ativa,0.0085,90.9
4,"R$75,30 M",Banco Genial,Galpões,PLOG11,0.0313,(21) 3923-3000,http://www.bancogenial.com/,36.501.198/0001-34,[email protected],2020-12-01 00:00:00,FII PLURAL L,0.75,Tijolo,2022-09-29,78.9,2907,83.95,100.4,750000,Renda Gestão Ativa,0.0095,65.41


In [22]:
df.head().dividend_yield

0    0.9093
1    0.9093
2    0.0085
3    0.0085
4    0.0095
Name: dividend_yield, dtype: float64

In [24]:
#ticker_data_dict = dataset.to_dict(orient='records')
df.to_dict(orient='records')

[{'patrimonio_liquido': 'R$1,36 M',
  'administrador': 'Oliveira Trust',
  'subtipo': 'Desenvolvimento',
  'ticker': 'LATR11B',
  'return_12_months': 0.8339,
  'telefone': '(21) 3514-0000',
  'site': 'www.oliveiratrust.com.br',
  'CNPJ': '17.209.378/0001-00',
  'email': '[email\xa0protected]',
  'registro_CVM': '2012-11-19 00:00:00',
  'name': 'FII LATERES',
  'ultimo_rendimento': Decimal('14.5'),
  'tipo': 'Tijolo',
  'date': '2022-09-29',
  'cotacao': Decimal('15.95'),
  'numero_cotistas': Decimal('166'),
  'max_52_weeks': Decimal('150'),
  'valor_patrimonial_por_cota': Decimal('10.98'),
  'numero_cotas': Decimal('124104'),
  'tipo_ANBIMA': 'Desenvolvimento para Venda Gestão Ativa',
  'dividend_yield': 0.9093,
  'min_52_weeks': Decimal('15.95')},
 {'patrimonio_liquido': 'R$1,36 M',
  'administrador': 'Oliveira Trust',
  'subtipo': 'Desenvolvimento',
  'ticker': 'LATR11B',
  'return_12_months': 0.8339,
  'telefone': '(21) 3514-0000',
  'site': 'www.oliveiratrust.com.br',
  'CNPJ': '17

In [ ]:
batch_write(dynamodb_resource, "fiis", ticker_data_dict, ['ticker', 'date'])

### Sending the ifix dataframe

In [ ]:
dataset = df_ifix

In [ ]:
dynamodb_resource = boto3.resource("dynamodb")
ifix_data_dict = dataset.to_dict(orient='records')
batch_write(dynamodb_resource, "ifix", ifix_data_dict)